In [1]:
import os

path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

In [75]:
from urllib.request import urlopen
import json
import numpy as np
import pandas as pd
from datetime import datetime, date
from tqdm.notebook import tqdm

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from app.utils.visualization import _plot_data
from app.utils.fetch_data import fred_fred, investing_api, alpha_vantage_api, FMP
from app.data_handling.feature_engineering import create_price_features

In [3]:
today = date.today().strftime("%Y-%m-%d")
start_date = '2010-01-01'
ticker = 'AAPL'

#### Fetch stock data

In [4]:
total_stock = investing_api('stock', ticker, start_date, today)

2022-07-22 00:30:14,846 [app.utils.fetch_data] [INFO] Fetching stock from investing_api: AAPL, from 01/01/2010 to 22/07/2022


In [5]:
# feature engineering
total_stock = create_price_features(total_stock)

In [6]:
total_stock.head(2)

,Date,Open,High,Low,Close,Volume,Currency,ticker,type,p_key,upper_shadow,lower_shadow,open2close,high2low,high2mean,low2mean,high2median,low2median
0,2010-01-04,7.62,7.66,7.58,7.64,493728704,USD,AAPL,stock,2010_01_04_AAPL,1.002618,1.005277,1.002625,1.010554,1.004590,0.994098,1.003932,0.993447
1,2010-01-05,7.66,7.70,7.62,7.66,601904640,USD,AAPL,stock,2010_01_05_AAPL,1.005222,1.005249,1.000000,1.010499,1.005222,0.994778,1.005222,0.994778


#### Fetch insider trading data

In [7]:
fmp = FMP()

2022-07-22 00:30:17,717 [app.utils.fetch_data] [INFO] Financial Modeling Prep api ready.


In [8]:
insider = fmp.get_historical_insider_trade_ticker(ticker, num_pages=35)
# filter with the according date
insider = insider.loc[insider['transactionDate']>=start_date]

2022-07-22 00:30:24,935 [app.utils.fetch_data] [INFO] Fetching 35 pages of AAPL insider trade data.


In [9]:
# cleaning some of the columns
insider['reportingName'] = insider['reportingName'].str.lower()

In [67]:
insider.tail(2)

,index,symbol,filingDate,transactionDate,reportingCik,transactionType,securitiesOwned,companyCik,reportingName,typeOfOwner,acquistionOrDisposition,formType,securitiesTransacted,price,securityName,link,price_adj,securitiesTransacted_adj
2272,72,AAPL,2010-02-04 17:43:55,2010-02-02,0001214107,G-Gift,10487.0,0000320193,oppenheimer peter,"officer: Senior Vice President, CFO",D,4,1250.0,0.00,Common Stock,https://www.sec.gov/Archives/edgar/data/000032...,0.000000,35000.0
2273,73,AAPL,2010-01-06 17:29:05,2010-01-04,0001252200,S-Sale,120781.0,0000320193,serlet bertrand,officer: Senior Vice President,D,4,10000.0,213.55,Common Stock,https://www.sec.gov/Archives/edgar/data/000032...,7.626786,280000.0


#### Adjust for stock split

In [55]:
stock_split = fmp.get_stock_split_history(ticker)
# only the relevant dates
stock_split = stock_split.loc[stock_split.date>=start_date]

2022-07-22 22:07:51,724 [app.utils.fetch_data] [INFO] Fetching stock split history for AAPL


In [58]:
stock_split

,date,label,numerator,denominator,symbol
0,2020-08-31,"August 31, 20",4.0,1.0,AAPL
1,2014-06-09,"June 09, 14",7.0,1.0,AAPL


In [53]:
# The stock price is already adjusted
# however, the insider data is not
stock_split = stock_split.sort_values(by=['date'], ascending=False)

insider['price_adj'] = insider['price']
insider['securitiesTransacted_adj'] = insider['securitiesTransacted']

for i, row in stock_split.iterrows():
    stock_multiplier = row['numerator']/row['denominator']
    adj_prices = insider.loc[insider['transactionDate']<=row['date'], 'price_adj'] / stock_multiplier
    adj_volume = insider.loc[insider['transactionDate']<=row['date'], 'securitiesTransacted_adj'] * stock_multiplier

    insider.loc[insider['transactionDate']<=row['date'], 'price_adj'] = adj_prices
    insider.loc[insider['transactionDate']<=row['date'], 'securitiesTransacted_adj'] = adj_volume

#### Create master dataset

In [11]:
# relevent columns from the insider dataset
insider_columns = ['transactionDate', 'transactionType', 'reportingName', 
                   'securitiesTransacted', 'price', 'securityName']

In [125]:
df = total_stock.merge(insider[insider_columns], left_on='Date', right_on='transactionDate')

In [63]:
df.head(2)

,Date,Open,High,Low,Close,Volume,Currency,ticker,type,p_key,...,high2mean,low2mean,high2median,low2median,transactionDate,transactionType,reportingName,securitiesTransacted,price,securityName
0,2010-01-04,7.62,7.66,7.58,7.64,493728704,USD,AAPL,stock,2010_01_04_AAPL,...,1.004590,0.994098,1.003932,0.993447,2010-01-04,S-Sale,serlet bertrand,10000.0,213.55,Common Stock
1,2010-02-02,7.00,7.01,6.91,6.99,698341952,USD,AAPL,stock,2010_02_02_AAPL,...,1.004658,0.990326,1.002144,0.987848,2010-02-02,G-Gift,oppenheimer peter,650.0,0.00,Common Stock


In [216]:
def _plot_data(plot_settings):
    """
    Create a plotly figure. The input should be a list of list.
    The inner list should be [dataset, x_column, y_column]

    Args:
        plot_settings(list): [[plot_type1, dataset, x_column, y_column, data_label], [plot_type1, dataset, x_column, y_column, data_label], ...]

    Returns:
        fig(plotly.graph_objs._figure.Figure): plot
    """

    fig = make_subplots(specs=[[{"secondary_y": True}]])

    for i, setting in enumerate(plot_settings):
        plot_type = setting[0]
        assert plot_type in ['scatter', 'line', 'bar']
        trace = _create_trace(plot_type, setting[1], setting[2], setting[3], setting[4], 'y'+str(i+1))
        fig.add_trace(trace, secondary_y=setting[5])
    
    # update fig to make it 
    update_dict = {}

    for i in range(len(plot_settings)-1):
        update_dict['yaxis'+str(i+2)] = {"overlaying":'y', 'visible': False}
    fig['layout'].update(update_dict)
    fig['layout'].update({'yaxis':{'visible':False}})
    
    fig.update_layout(legend=dict(
                        orientation="h",
                        yanchor="bottom",
                        y=1.02,
                        xanchor="right",
                        x=1))

    return fig

In [219]:
a = _plot_data([['line', df, 'Date', 'Close', 'AAPL', False], 
                ['scatter', insider, 'transactionDate', 'price_adj', 'insider data', False]])
a

In [218]:
a = _plot_data([['line', df, 'Date', 'Close', 'AAPL', False], 
                ['scatter', insider, 'transactionDate', 'price_adj', 'insider data', True]])
a

In [205]:
tuple(a)

(Scatter({
     'name': 'AAPL',
     'x': array([datetime.datetime(2010, 1, 4, 0, 0),
                 datetime.datetime(2010, 2, 2, 0, 0),
                 datetime.datetime(2010, 2, 2, 0, 0), ...,
                 datetime.datetime(2022, 5, 4, 0, 0),
                 datetime.datetime(2022, 5, 4, 0, 0),
                 datetime.datetime(2022, 5, 6, 0, 0)], dtype=object),
     'y': array([  7.64,   6.99,   6.99, ..., 166.02, 166.02, 157.28]),
     'yaxis': 'y'
 }),
 Scatter({
     'mode': 'markers',
     'name': 'insider data',
     'x': array([datetime.datetime(2022, 5, 6, 0, 0),
                 datetime.datetime(2022, 5, 4, 0, 0),
                 datetime.datetime(2022, 5, 4, 0, 0), ...,
                 datetime.datetime(2010, 2, 2, 0, 0),
                 datetime.datetime(2010, 2, 2, 0, 0),
                 datetime.datetime(2010, 1, 4, 0, 0)], dtype=object),
     'y': array([  0.        , 165.95      , 165.06      , ...,   0.        ,
                   0.        ,   7.626785

In [202]:
fig = go.Figure(tuple(a))

In [203]:
fig

In [210]:
trace1 = _create_trace('line', df, 'Date', 'Close', 'AAPL', 'y1')
trace3 = _create_trace('line', df, 'Date', 'Open', 'AAPL', 'y1')
trace2 = _create_trace('scatter', insider, 'transactionDate', 'price_adj', 'insider data', 'y2')

In [211]:
trace_list = tuple([trace1, trace2])

In [213]:
trace_list

(Scatter({
     'name': 'AAPL',
     'x': array([datetime.datetime(2010, 1, 4, 0, 0),
                 datetime.datetime(2010, 2, 2, 0, 0),
                 datetime.datetime(2010, 2, 2, 0, 0), ...,
                 datetime.datetime(2022, 5, 4, 0, 0),
                 datetime.datetime(2022, 5, 4, 0, 0),
                 datetime.datetime(2022, 5, 6, 0, 0)], dtype=object),
     'y': array([  7.64,   6.99,   6.99, ..., 166.02, 166.02, 157.28]),
     'yaxis': 'y'
 }),
 Scatter({
     'mode': 'markers',
     'name': 'insider data',
     'x': array([datetime.datetime(2022, 5, 6, 0, 0),
                 datetime.datetime(2022, 5, 4, 0, 0),
                 datetime.datetime(2022, 5, 4, 0, 0), ...,
                 datetime.datetime(2010, 2, 2, 0, 0),
                 datetime.datetime(2010, 2, 2, 0, 0),
                 datetime.datetime(2010, 1, 4, 0, 0)], dtype=object),
     'y': array([  0.        , 165.95      , 165.06      , ...,   0.        ,
                   0.        ,   7.626785

In [212]:
go.Figure(trace_list)

In [214]:
a = make_subplots(specs=[[{"secondary_y": True}]])
a = a.add_trace(trace1)
a = a.add_trace(trace2, secondary_y=False)
a = a.add_trace(trace3)

In [215]:
a

In [148]:
fig1 = px.line(df, x="Date", y="Close")
fig1.update_traces(line=dict(color = 'red'))

fig2 = px.scatter(insider, x="transactionDate", y="price_adj")

fig3 = go.Figure(data=fig1.data + fig2.data)
fig3.show()

In [149]:
fig1.data

(Scattergl({
     'hovertemplate': 'Date=%{x}<br>Close=%{y}<extra></extra>',
     'legendgroup': '',
     'line': {'color': 'red', 'dash': 'solid'},
     'marker': {'symbol': 'circle'},
     'mode': 'lines',
     'name': '',
     'showlegend': False,
     'x': array([datetime.datetime(2010, 1, 4, 0, 0),
                 datetime.datetime(2010, 2, 2, 0, 0),
                 datetime.datetime(2010, 2, 2, 0, 0), ...,
                 datetime.datetime(2022, 5, 4, 0, 0),
                 datetime.datetime(2022, 5, 4, 0, 0),
                 datetime.datetime(2022, 5, 6, 0, 0)], dtype=object),
     'xaxis': 'x',
     'y': array([  7.64,   6.99,   6.99, ..., 166.02, 166.02, 157.28]),
     'yaxis': 'y'
 }),)

In [150]:
fig2.data

(Scattergl({
     'hovertemplate': 'transactionDate=%{x}<br>price_adj=%{y}<extra></extra>',
     'legendgroup': '',
     'marker': {'color': '#636efa', 'symbol': 'circle'},
     'mode': 'markers',
     'name': '',
     'showlegend': False,
     'x': array([datetime.datetime(2022, 5, 6, 0, 0),
                 datetime.datetime(2022, 5, 4, 0, 0),
                 datetime.datetime(2022, 5, 4, 0, 0), ...,
                 datetime.datetime(2010, 2, 2, 0, 0),
                 datetime.datetime(2010, 2, 2, 0, 0),
                 datetime.datetime(2010, 1, 4, 0, 0)], dtype=object),
     'xaxis': 'x',
     'y': array([  0.        , 165.95      , 165.06      , ...,   0.        ,
                   0.        ,   7.62678571]),
     'yaxis': 'y'
 }),)

In [151]:
fig1.data + fig2.data

(Scattergl({
     'hovertemplate': 'Date=%{x}<br>Close=%{y}<extra></extra>',
     'legendgroup': '',
     'line': {'color': 'red', 'dash': 'solid'},
     'marker': {'symbol': 'circle'},
     'mode': 'lines',
     'name': '',
     'showlegend': False,
     'x': array([datetime.datetime(2010, 1, 4, 0, 0),
                 datetime.datetime(2010, 2, 2, 0, 0),
                 datetime.datetime(2010, 2, 2, 0, 0), ...,
                 datetime.datetime(2022, 5, 4, 0, 0),
                 datetime.datetime(2022, 5, 4, 0, 0),
                 datetime.datetime(2022, 5, 6, 0, 0)], dtype=object),
     'xaxis': 'x',
     'y': array([  7.64,   6.99,   6.99, ..., 166.02, 166.02, 157.28]),
     'yaxis': 'y'
 }),
 Scattergl({
     'hovertemplate': 'transactionDate=%{x}<br>price_adj=%{y}<extra></extra>',
     'legendgroup': '',
     'marker': {'color': '#636efa', 'symbol': 'circle'},
     'mode': 'markers',
     'name': '',
     'showlegend': False,
     'x': array([datetime.datetime(2022, 5, 6, 0, 

In [175]:
traces = [list(fig1.data), list(fig2.data)]

In [162]:
import itertools

In [176]:
trace_list_flat = list(itertools.chain(*traces))

In [177]:
go.Figure(tuple(trace_list_flat))

In [157]:
[traces[0].append(i) for i in ]

(Scattergl({
     'hovertemplate': 'Date=%{x}<br>Close=%{y}<extra></extra>',
     'legendgroup': '',
     'line': {'color': 'red', 'dash': 'solid'},
     'marker': {'symbol': 'circle'},
     'mode': 'lines',
     'name': '',
     'showlegend': False,
     'x': array([datetime.datetime(2010, 1, 4, 0, 0),
                 datetime.datetime(2010, 2, 2, 0, 0),
                 datetime.datetime(2010, 2, 2, 0, 0), ...,
                 datetime.datetime(2022, 5, 4, 0, 0),
                 datetime.datetime(2022, 5, 4, 0, 0),
                 datetime.datetime(2022, 5, 6, 0, 0)], dtype=object),
     'xaxis': 'x',
     'y': array([  7.64,   6.99,   6.99, ..., 166.02, 166.02, 157.28]),
     'yaxis': 'y'
 }),)

In [161]:
list(fig1.data) + list(fig2.data)

[Scattergl({
     'hovertemplate': 'Date=%{x}<br>Close=%{y}<extra></extra>',
     'legendgroup': '',
     'line': {'color': 'red', 'dash': 'solid'},
     'marker': {'symbol': 'circle'},
     'mode': 'lines',
     'name': '',
     'showlegend': False,
     'x': array([datetime.datetime(2010, 1, 4, 0, 0),
                 datetime.datetime(2010, 2, 2, 0, 0),
                 datetime.datetime(2010, 2, 2, 0, 0), ...,
                 datetime.datetime(2022, 5, 4, 0, 0),
                 datetime.datetime(2022, 5, 4, 0, 0),
                 datetime.datetime(2022, 5, 6, 0, 0)], dtype=object),
     'xaxis': 'x',
     'y': array([  7.64,   6.99,   6.99, ..., 166.02, 166.02, 157.28]),
     'yaxis': 'y'
 }),
 Scattergl({
     'hovertemplate': 'transactionDate=%{x}<br>price_adj=%{y}<extra></extra>',
     'legendgroup': '',
     'marker': {'color': '#636efa', 'symbol': 'circle'},
     'mode': 'markers',
     'name': '',
     'showlegend': False,
     'x': array([datetime.datetime(2022, 5, 6, 0, 

In [159]:
list(traces[0]).append(traces[1])

In [143]:
trace1 = _create_trace('line', df, 'Date', 'Close', 'name', 'y')

In [144]:
trace1

Scatter({
    'name': 'name',
    'x': array([datetime.datetime(2010, 1, 4, 0, 0),
                datetime.datetime(2010, 2, 2, 0, 0),
                datetime.datetime(2010, 2, 2, 0, 0), ...,
                datetime.datetime(2022, 5, 4, 0, 0),
                datetime.datetime(2022, 5, 4, 0, 0),
                datetime.datetime(2022, 5, 6, 0, 0)], dtype=object),
    'y': array([  7.64,   6.99,   6.99, ..., 166.02, 166.02, 157.28]),
    'yaxis': 'y'
})

In [ ]:
price_adj

In [39]:
# use the stock split date from latest to oldest
# compare the date with the stock dates

    df.loc[df['Date']<=row['date'], 'applied_date'] = row['date']
    
# merge to columns so that we know which numerator and denominator we need to apply
df = pd.merge(df, stock_split, left_on='applied_date', right_on='date', how='left')
# create adjusted prices and adjusted volume


In [40]:
df

,Date,Open,High,Low,Close,Volume,Currency,ticker,type,p_key,...,reportingName,securitiesTransacted,price,securityName,applied_date,date,label,numerator,denominator,symbol
0,2010-01-04,7.62,7.66,7.58,7.64,493728704,USD,AAPL,stock,2010_01_04_AAPL,...,serlet bertrand,10000.0,213.5500,Common Stock,2014-06-09,2014-06-09,"June 09, 14",7.0,1.0,AAPL
1,2010-02-02,7.00,7.01,6.91,6.99,698341952,USD,AAPL,stock,2010_02_02_AAPL,...,oppenheimer peter,650.0,0.0000,Common Stock,2014-06-09,2014-06-09,"June 09, 14",7.0,1.0,AAPL
2,2010-02-02,7.00,7.01,6.91,6.99,698341952,USD,AAPL,stock,2010_02_02_AAPL,...,oppenheimer peter,1250.0,0.0000,Common Stock,2014-06-09,2014-06-09,"June 09, 14",7.0,1.0,AAPL
3,2010-02-23,7.14,7.19,6.99,7.04,575313472,USD,AAPL,stock,2010_02_23_AAPL,...,drexler millard s,20000.0,198.4257,Common Stock,2014-06-09,2014-06-09,"June 09, 14",7.0,1.0,AAPL
4,2010-02-25,7.05,7.24,7.03,7.21,665396928,USD,AAPL,stock,2010_02_25_AAPL,...,gore albert jr,990.0,NaN,Restricted Stock Unit,2014-06-09,2014-06-09,"June 09, 14",7.0,1.0,AAPL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853,2022-05-04,159.67,166.48,159.26,166.02,108256504,USD,AAPL,stock,2022_05_04_AAPL,...,adams katherine l.,900.0,162.7700,Common Stock,NaT,NaT,NaN,NaN,NaN,NaN
1854,2022-05-04,159.67,166.48,159.26,166.02,108256504,USD,AAPL,stock,2022_05_04_AAPL,...,adams katherine l.,4006.0,161.9300,Common Stock,NaT,NaT,NaN,NaN,NaN,NaN
1855,2022-05-04,159.67,166.48,159.26,166.02,108256504,USD,AAPL,stock,2022_05_04_AAPL,...,adams katherine l.,5316.0,160.8200,Common Stock,NaT,NaT,NaN,NaN,NaN,NaN
1856,2022-05-04,159.67,166.48,159.26,166.02,108256504,USD,AAPL,stock,2022_05_04_AAPL,...,adams katherine l.,9136.0,159.9400,Common Stock,NaT,NaT,NaN,NaN,NaN,NaN


In [34]:
df.applied_date

0       2014-06-09 00:00:00
1       2014-06-09 00:00:00
2       2014-06-09 00:00:00
3       2014-06-09 00:00:00
4       2014-06-09 00:00:00
               ...         
1853                    NaN
1854                    NaN
1855                    NaN
1856                    NaN
1857                    NaN
Name: applied_date, Length: 1858, dtype: object

In [ ]:
pd.merge()

In [ ]:
def adjust_stock_split(stock_df, stock_df_date, split_df,
                       split_df_date, split_df_numer, split_df_denom):
    
    
    stock_df
    
    split_df = split_df.sort_values(by=[split_df_date], ascending=False)
    
    

In [ ]:
stock_split.date

In [14]:
stock_split = fmp.get_stock_split_history(ticker)
# data after start date
stock_split = stock_split.loc[stock_split.date > start_date]

2022-07-22 00:31:54,714 [app.utils.fetch_data] [INFO] Fetching stock split history for AAPL


#### Train and test, and validation

In [ ]:
validation = 

In [ ]:
# only common stocks.
# Restricted stock units are special kind of stocks that is given to the employee under certain conditions

In [112]:
volume_by_AD = insider.groupby(['transactionDate_Y_M', 'acquistionOrDisposition']).agg({"securitiesTransacted":"sum"}).reset_index()

volume_by_AD.head(2)

,transactionDate_Y_M,acquistionOrDisposition,securitiesTransacted
0,2016-04,A,219268.0
1,2016-04,D,352435.0


In [116]:
dates = volume_by_AD.transactionDate_Y_M.drop_duplicates()

acquisition = volume_by_AD.loc[volume_by_AD.acquistionOrDisposition=='A']
acquisition = pd.merge(dates, acquisition, how='left')
acquisition['acquistionOrDisposition'] = acquisition['acquistionOrDisposition'].fillna('A')
acquisition['securitiesTransacted'] = acquisition['securitiesTransacted'].fillna(0)

disposition = volume_by_AD.loc[volume_by_AD.acquistionOrDisposition=='D']
disposition = pd.merge(dates, disposition, how='left')
disposition['acquistionOrDisposition'] = disposition['acquistionOrDisposition'].fillna('D')
disposition['securitiesTransacted'] = disposition['securitiesTransacted'].fillna(0)

In [117]:
disposition.head(2)

,transactionDate_Y_M,acquistionOrDisposition,securitiesTransacted
0,2016-04,D,352435.0
1,2016-05,D,519946.0


In [118]:
acquisition.tail(3)

,transactionDate_Y_M,acquistionOrDisposition,securitiesTransacted
58,2022-03,A,13480.0
59,2022-04,A,620558.0
60,2022-05,A,0.0


In [122]:
fig = go.Figure()

trace1 = go.Bar(x=dates,
                y=acquistion.securitiesTransacted,
                name="Acquisition",
                marker_color='indianred')
trace2 = go.Bar(x=dates,
                y=disposition.securitiesTransacted,
                marker_color='blue',
                name="Disposition")

fig.add_trace(trace1)
fig.add_trace(trace2)

fig.update_layout(barmode='group')

In [66]:
list(insider.link)[0]

'https://www.sec.gov/Archives/edgar/data/0000320193/000032019322000063/0000320193-22-000063-index.htm'

In [ ]:
trace = go.Bar(x=data[x],
               y=data[y],
               name=name,
               yaxis=yaxis)

In [ ]:
# disposition and acquisition per date

In [ ]:
go.Bar()

In [50]:
insider.columns

Index(['symbol', 'filingDate', 'transactionDate', 'reportingCik',
       'transactionType', 'securitiesOwned', 'companyCik', 'reportingName',
       'typeOfOwner', 'acquistionOrDisposition', 'formType',
       'securitiesTransacted', 'price', 'securityName', 'link'],
      dtype='object')

In [52]:
insider.transactionType.unique()

array(['G-Gift', 'S-Sale', 'M-Exempt', 'F-InKind', 'A-Award'],
      dtype=object)

In [53]:
insider.reportingName.unique()

array(['BELL JAMES A', 'Adams Katherine L.', "O'BRIEN DEIRDRE",
       'KONDO CHRIS', 'WILLIAMS JEFFREY E', 'Maestri Luca',
       'WAGNER SUSAN', 'SUGAR RONALD D', 'LOZANO MONICA C',
       'LEVINSON ARTHUR D', 'GORE ALBERT JR', 'Gorsky Alex',
       'JUNG ANDREA', 'COOK TIMOTHY D', 'IGER ROBERT A',
       'AHRENDTS ANGELA J', 'Srouji Johny', 'FEDERIGHI CRAIG',
       'Riccio Daniel J.', 'SCHILLER PHILIP W', 'Cue Eduardo H',
       'SEWELL D BRUCE'], dtype=object)

In [54]:
insider.typeOfOwner.unique()

array(['director', 'officer: SVP, GC and Secretary',
       'officer: Senior Vice President',
       'officer: Principal Accounting Officer', 'officer: COO',
       'officer: Senior Vice President, CFO',
       'director, officer: Chief Executive Officer',
       'officer: SVP, General Counsel',
       "officer: SVP, Gen'l Counsel, Secretary"], dtype=object)

In [60]:
insider[['reportingName', 'typeOfOwner']].drop_duplicates().sort_values(by='reportingName')

,reportingName,typeOfOwner
22,AHRENDTS ANGELA J,officer: Senior Vice President
1,Adams Katherine L.,"officer: SVP, GC and Secretary"
85,Adams Katherine L.,"officer: SVP, General Counsel"
0,BELL JAMES A,director
5,COOK TIMOTHY D,"director, officer: Chief Executive Officer"
46,Cue Eduardo H,officer: Senior Vice President
59,FEDERIGHI CRAIG,officer: Senior Vice President
45,GORE ALBERT JR,director
46,Gorsky Alex,director
92,IGER ROBERT A,director


In [55]:
insider.acquistionOrDisposition.unique()

array(['D', 'A'], dtype=object)

In [56]:
insider.formType.unique()

array(['4'], dtype=object)

In [57]:
insider.securityName.unique()

array(['Common Stock', 'Restricted Stock Unit',
       'Director Stock Option (Right to Buy)', 'Restricted Stock Units'],
      dtype=object)